In [173]:
def returnProjectIDs():
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT projectid, sum(utmvisits) FROM allinprojectstatistic GROUP BY projectid")
    result = []
    rows = cur.fetchall()
    projects = pd.DataFrame(rows)
    projects.columns = [desc[0] for desc in cur.description]
    projects = projects[projects['sum'] > 0].reset_index(drop=True).sort_values(by=['projectid'])
    projects = projects[~projects['projectid'].isin([1,5])].reset_index(drop=True)
    projectIDs = projects['projectid'].unique()
    return projectIDs
def returnTargetIDsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    if len(rows) > 0:
        targets = pd.DataFrame(rows)
        targets.columns = [desc[0] for desc in cur.description]
        return targets
    else:
        return []
def returnProjectTargetStat(projectID,targetIDsStr):
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT id, projectid, targetid, date, goals, medium, brokerid FROM allinprojecttargets WHERE projectid = {0} AND targetid IN ({1})".format(projectID,targetIDsStr))
    result = []
    rows = cur.fetchall()
    targetStat = pd.DataFrame(rows)
    targetStat.columns = [desc[0] for desc in cur.description]
    return targetStat
def returnDetailStat(projectID):
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT id, projectid, date, goalcompletions, utmvisits, medium, brokerid FROM allinprojectstatisticdetail WHERE projectid=%s", [projectID])
    result = []
    rows = cur.fetchall()
    detailStat = pd.DataFrame(rows)
    detailStat.columns = [desc[0] for desc in cur.description]
    return detailStat
def returnPaymentStat(projectID):
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT id, projectid, date, brokerid, sum FROM allinprojectstatisticpayment WHERE projectid=%s", [projectID])
    result = []
    rows = cur.fetchall()
    paymentStat = pd.DataFrame(rows)
    paymentStat.columns = [desc[0] for desc in cur.description]
    return paymentStat

In [200]:
projectID = 2381409
targetIDs = returnTargetIDsByProjectID(projectID)
if len(targetIDs) > 0:
    targetIDsStr = ','.join([str(targetid) for targetid in targetIDs['targetid'].unique()])
    try:
        detailStatistics = returnDetailStat(projectID)
    except:
        print 'Error while getting detailed project statistics'
    try:
        targetStatistics = returnProjectTargetStat(projectID,targetIDsStr)[['date','goals','medium','brokerid']]
    except:
        print 'Error while getting target project statistics'
    targetStatistics = targetStatistics.groupby(by=['date','brokerid'],as_index=False).sum()
    result = pd.merge(detailStatistics, targetStatistics, on=['date', 'brokerid'], how='outer').fillna(0)
    try:
        paymentStatistics = returnPaymentStat(projectID)[['date','brokerid','sum']]
    except:
        print 'Error while getting payment project statistics'
    result2 = pd.merge(result, paymentStatistics, on=['date', 'brokerid'], how='left').fillna(0)

In [205]:
result2

,id,projectid,date,goalcompletions,utmvisits,medium,brokerid,goals,sum
0,1082524,2381409,2016-02-21,0,32,organic,0,0,0
1,1082525,2381409,2016-02-21,0,1,direct,18,0,0.000000
2,1082526,2381409,2016-02-21,0,17,rtb,28,0,63.000000
3,1082527,2381409,2016-02-21,0,1,direct_context,2000,0,0.000000
4,958799,2381409,2016-01-22,0,80,organic,0,0,0
5,958800,2381409,2016-01-22,0,130,rtb,28,0,414.000000
6,958801,2381409,2016-01-22,0,36,direct,18,0,617.850000
7,958802,2381409,2016-01-22,0,8,direct_context,2000,0,423.860000
8,958803,2381409,2016-01-22,0,2,direct_search,2001,0,193.990000
9,745163,2381409,2015-11-24,1,44,direct,18,1,794.030000


In [211]:
dataframes = []
projectIDs = returnProjectIDs()
for projectID in projectIDs:
    projectID = int(projectID)
    print 'Proccessing project #{0}'.format(projectID)
    targetIDs = returnTargetIDsByProjectID(projectID)
    if len(targetIDs) > 0:
        targetIDsStr = ','.join([str(targetid) for targetid in targetIDs['targetid'].unique()])
        try:
            print 'Getting detailed project statistics'
            detailStatistics = returnDetailStat(projectID)
        except:
            print 'Error while getting detailed project statistics'
            continue
        try:
            print 'Getting target project statistics'
            targetStatistics = returnProjectTargetStat(projectID,targetIDsStr)[['date','goals','medium','brokerid']]
        except:
            print 'Error while getting target project statistics'
            continue
        targetStatistics = targetStatistics.groupby(by=['date','brokerid'],as_index=False).sum()
        result = pd.merge(detailStatistics, targetStatistics, on=['date', 'brokerid'], how='outer').fillna(0)
        try:
            print 'Getting payment project statistics'
            paymentStatistics = returnPaymentStat(projectID)[['date','brokerid','sum']]
        except:
            print 'Error while getting payment project statistics'
            continue
        result2 = pd.merge(result, paymentStatistics, on=['date', 'brokerid'], how='left').fillna(0)
        if len(result2) > 0:
            print 'Result length = {0}'.format(len(result2))
            dataframes.append(result2)
        else:
            print 'Result dataframe length = 0'
    else:
        print 'No targets'

Proccessing project #2214297
Getting detailed project statistics
Getting target project statistics
Getting payment project statistics
Result length = 2482
Proccessing project #2226096
No targets
Proccessing project #2228439
Getting detailed project statistics
Getting target project statistics
Getting payment project statistics
Result length = 780
Proccessing project #2229285
Getting detailed project statistics
Getting target project statistics
Getting payment project statistics
Result length = 1749
Proccessing project #2232213
No targets
Proccessing project #2234924
Getting detailed project statistics
Getting target project statistics
Getting payment project statistics
Result length = 1895
Proccessing project #2235000
No targets
Proccessing project #2235683
Getting detailed project statistics
Getting target project statistics
Getting payment project statistics
Result length = 2024
Proccessing project #2235990
Getting detailed project statistics
Getting target project statistics
Getting

In [228]:
allData.dtypes
allData['id'] = allData['id'].astype(int)
allData['projectid'] = allData['projectid'].astype(int)

In [229]:
allData.dtypes

id                   int32
projectid            int32
date                object
goalcompletions    float64
utmvisits          float64
medium              object
brokerid           float64
goals              float64
sum                 object
dtype: object

In [230]:
allData.to_csv('D:\\allProjectsData.csv',sep='\t',encoding='utf8',decimal=',')

In [218]:
dataframes[0]

,id,projectid,date,goalcompletions,utmvisits,medium,brokerid,goals,sum
0,1158893,2214297,2016-03-11,0,61,direct,18,0,238.250000
1,1158894,2214297,2016-03-11,0,64,rtb,28,0,213.000000
2,1158895,2214297,2016-03-11,0,12,organic,0,0,0
3,1158896,2214297,2016-03-11,0,2,acontext,1016,0,0
4,1158897,2214297,2016-03-11,0,57,direct_search,2001,0,238.250000
5,1162402,2214297,2016-03-15,0,14,rtb,28,0,180.000000
6,1162403,2214297,2016-03-15,0,2,organic,0,0,0
7,1162404,2214297,2016-03-15,0,6,direct,18,0,151.740000
8,1162405,2214297,2016-03-15,0,5,direct_search,2001,0,151.740000
9,1162406,2214297,2016-03-13,0,82,direct,18,0,263.850000
